# Power Point Presentation maker

## Configs
Load the Api keys from the environment, and create some basic util functions

In [ ]:
from __future__ import annotations

import os

from dotenv import load_dotenv

load_dotenv()
api_key = os.environ.get("OPENAI_API_KEY")
if not api_key:
    msg = "Missing API_KEY"
    raise OSError(msg)

In [ ]:
import string


def remove_punctuation(text: str) -> str:
    translator = str.maketrans("", "", string.punctuation)
    return text.translate(translator)

## Using OpenAI chat with structured Format to get the PPT content

[OpenAI docs - structured format](https://platform.openai.com/docs/guides/structured-outputs)
[Pydantic](https://docs.pydantic.dev/latest/) is being used to pass a Structured Format to OpenAI in order to receive correctly formatted response. 

In [ ]:
from pydantic import BaseModel


class SingleSlide(BaseModel):
    Slide_Number: int
    Slide_Title: str
    Slide_Text: str
    Image_Prompt: str


class ResponseFormat(BaseModel):
    slide: list[SingleSlide]


class ResponseFormatBase(BaseModel):
    text: str

In [ ]:
from typing import TypeVar

from pydantic import ValidationError

T = TypeVar("T", bound=BaseModel)


def validate_response(response: T | None, cls: type[T]) -> T:
    try:
        return cls.model_validate(response)
    except ValidationError as e:
        print(f"Error Validating response -> {response} \n {e}")
        raise

In [ ]:
from openai import OpenAI


def chat_completion(prompt: str, response_format: type[T]) -> T | None:
    client = OpenAI(api_key=api_key)
    completion = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "user", "content": prompt},
        ],
        response_format=response_format,
    )
    return completion.choices[0].message.parsed

## Prompts
2 Prompts are being used to generate the response, the first generates a PPT outline, and the second uses that outline to generate the PPT content

In [ ]:
from prompts import first_prompt, second_prompt

first_chat = validate_response(
    chat_completion(first_prompt, ResponseFormatBase),
    ResponseFormatBase,
)
second_chat = validate_response(
    chat_completion(second_prompt + "# Outline\n" + first_chat.text, ResponseFormat),
    ResponseFormat,
)

In [ ]:
for i in second_chat.slide:
    print(f"{i.Slide_Number}->{i.Slide_Title}\n{i.Slide_Text}\n{i.Image_Prompt}\n")

## Function to create images with Stable Diffusion

Using a local Stable Diffusion API to create the images, currently using the [WebUI Forge API](https://github.com/lllyasviel/stable-diffusion-webui-forge).

In [ ]:
import base64
from pathlib import Path

import requests

# Define the URL and the payload to send.
url = "http://127.0.0.1:7860"


def imagecreator(prompt: str, name: str, width: int, height: int) -> None:
    """Access local Stable Diffusion to create images.

    Args:
        prompt (str): prompt for image creation
        name (str): Name of the resulting image
        width (int): Image Width
        height (int): Image Height

    """
    payload = {
        "prompt": prompt,
        "styles": ["Avant-garde"],
        "steps": 20,
        "sampler_name": "DPM++ 2M",
        "scheduler": "Beta",
        "cfg_scale": 1,
        "width": width,
        "height": height,
    }

    # Send said payload to said URL through the API.
    response = requests.post(url=f"{url}/sdapi/v1/txt2img", json=payload, timeout=300)
    r = response.json()

    # Decode and save the image.
    with Path(f"images/{name}.png").open("wb") as f:
        f.write(base64.b64decode(r["images"][0]))

## Create a PPTX with the responses generated with ChatGPT and the Stable Diffusion images

Using the [Python-pptx](https://python-pptx.readthedocs.io/en/latest/index.html) library for PowerPoint creation

Check Available Layouts

In [ ]:
from pptx import Presentation

prs = Presentation()
for idx, layout in enumerate(prs.slide_layouts):
    print(f"{idx}: {layout.name}")

# PPT Creation
The Previous OpenAI chat returned a Title, Text and a prompt for image creation
This information is being added to basic empty slides

## Manual Step
After the PPT is created, the PPT is getting manually opened and the slides are organized using the designer option inside the software

In [ ]:
from pptx import Presentation
from pptx.util import Inches

prs = Presentation()
title_slide_layout = prs.slide_layouts[0]

for ai_slide in second_chat.slide:
    slide = prs.slides.add_slide(title_slide_layout)

    # Remove Punctuation from title
    slide_title = remove_punctuation(ai_slide.Slide_Title)

    # Add background image
    left = top = Inches(0)
    slide_width = prs.slide_width
    slide_height = prs.slide_height
    # Create Image
    imagecreator(
        ai_slide.Image_Prompt,
        slide_title,
        955,
        537,
    )
    # Add Image to slide
    pic = slide.shapes.add_picture(
        str(Path(f"images/{slide_title}.png")),
        left,
        top,
        slide_width,
        slide_height,
    )

    # Add Text
    title_shape = slide.shapes.title
    if title_shape is None:
        msg = "Shape is None"
        raise ValueError(msg)
    title_shape.text = ai_slide.Slide_Title

    body_shape = slide.shapes.placeholders[1]
    tf = body_shape.text_frame  # type: ignore[reportAttributeAccessIssue]
    tf.text = ai_slide.Slide_Text


prs.save("test.pptx")